In [197]:
import requests
from datetime import date, timedelta
import time
from pprint import pprint
import json
import os
import pandas as pd 
import bs4 as bs
import requests
import yfinance as yf
import datetime
from datetime import datetime as dt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

SUBMIT_URL = 'https://informationtracer.com/submit'
STATUS_URL = 'https://informationtracer.com/status'
RESULT_URL = 'https://informationtracer.com/result'


def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    return score
# STEP 1: submit a search query, and get id_hash256 (a unique identifier)
def step_1_submit(query, token, start_date, end_date):
    id_hash256 = None
    try:
        response = requests.post(SUBMIT_URL, 
                             timeout=10,
                             json={'query': query, 
                                   'token': token,
                                   'start_date': start_date,
                                    'end_date': end_date,
                                    'twitter_only': "true"
                                   }                                   
                            )
        if 'id_hash256' in response.json():
            id_hash256 = response.json()['id_hash256']
        else:
            print("Submission failed!")
            print(response.json())

    except Exception as e:
        print("Submission failed!")
        print(e)
    
    return id_hash256
        

# STEP 2: periodically check status, and get partial results (first 5 tweets)
def step_2_check_status(id_hash256, token):
    task_status = None
    MAX_ROUND = 40
    current_round = 0
    include_partial_results = 1

    while task_status != 'finished' and  current_round < MAX_ROUND:            
        current_round += 1
#         print(current_round)
        try:
            full_url = '{}?id_hash256={}&token={}&include_partial_results={}'.format(
                STATUS_URL, id_hash256, token, include_partial_results
                )
            response = requests.get(full_url, timeout=10).json()            
#             print(response)
            if 'status' in response:
                # NOTE: can render status_percentage, status_text on the UI
                task_status = response['status']
                status = response['status']
                status_percentage = response['status_percentage']
                status_text = response['status_text']
                tweet_preview = response.get('tweet_preview', None)
#                 if tweet_preview:
                    # NOTE: can render tweet_preview on the UI            
#                     print('received {} partial tweets'.format(len(tweet_preview)))
                    # TO save bandwidth, we can tell the API not to return partial results
#                     include_partial_results = 0

                if status != 'finished':
                    time.sleep(6)
                else:
                    return 'finished'

            else:
                print('status is not in response')
                pprint(response)
                return 'failed'

        except Exception as e:
            print('Exception when checking job status!')
            print(e)

    return 'timeout'


# STEP 3: get full results
def step_3_get_result(id_hash256, token):
    try:
        response = requests.get('{}?token={}&id_hash256={}'.format(RESULT_URL, token, id_hash256), timeout=10)
#         pprint(response.json().keys())
#         pprint(response.json())
        return response.json()
    except Exception as e:
        print(e)

    return []

def step_b1_loadTicker():
    try: 
        resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
        soup = bs.BeautifulSoup(resp.text, 'lxml')
        table = soup.find('table', {'class': 'wikitable sortable'})
        table.findAll('tr')[1:][0].findAll('td')[1].text
    except: 
        print("cannot get the table from Wiki!")
    tickers = []
    companies = []
    dc = {}
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        company = row.findAll('td')[1].text
        tickers.append(ticker)
        companies.append(company)
        dc[ticker.replace('\n', '').replace('.', '-')] = company
    tickers = [s.replace('\n', '').replace('.', '-') for s in tickers]
    start =  datetime.datetime(2023, 12, 1) - timedelta(days = 7)
    end = datetime.datetime(2023, 12, 1)
    data = yf.download(tickers, start=start, end=end)
    df_stock = data.stack().reset_index().rename(index=str, columns={"level_1": "Symbol"}).sort_values(['Symbol','Date'])
#     df_stock.set_index('Date', inplace=True)
    listCompany = [dc[c] for c in df_stock['Symbol']]
    df_stock['company'] = listCompany
    return df_stock.reset_index(drop = True)


In [308]:
# assume that if there is a buy signal, we buy 1 share;
# if there is a sell signal, we sell 1 share; 
# if there is a buy and a sell then we realize the pnl, else we take the last day as the day to exit and calculate the pnl 

#get the implied vols!! I do not know how 
def calculate_total_pnl(df_stock, n):
    pnl = 0 
    stock_pnl = []
    stock_names = []
    len_tweets = []


    for i in range(0,len(df_stock['Symbol']),n): 
#         print("Symbol", df_stock['Symbol'][i])
        symbol = df_stock['Symbol'][i]
        temp_hold = 0
        long = False
        short = False
        total_pnl = 0 
        total_t = 0
        num = 0 
        num_s = 0 
        for j in range(n):

#             print(df_stock['Symbol'][i+j], num)
            try:
                num_tw = len(df_stock['Posts_t'][i+j][1:-1].split("},"))
            except:
                print(df_stock['Symbol'][i], df_stock['Posts_t'][i+j])
                num_tw = 0 
            if num_tw >= 1 and len(df_stock['Posts_t'][i+j][1:-1].split("},")[0])>0:
                total_t +=num_tw
            else:
                total_t += 0
#             total_t += len(df_stock['Posts_t'][i][1:-1].split("},"))
#             if symbol != df_stock['Symbol'][i+j]:
#                 print(i, df_stock['Symbol'][i+j])
    #         print("j",j)
    #         print(df_stock['Symbol'][i+j], "Vader:",df_stock['Sentiment_Vader'][i+j])
            if df_stock['Sentiment_Vader'][i+j] < 0.01 and df_stock['Sentiment_Vader'][i+j] > -0.05:
    #             print('pass')

    #                 print('short: buy to cover on Friday!', 'Entry price', temp_hold, 'Exit price',df_stock['Adj Close'][i+j])
                pass
            else:
    #             print(df_stock['Sentiment_Vader'][i+j]) 
                if df_stock['Sentiment_Vader'][i+j] >= 0.05:
                    # no position
                    if not long and not short:  # the first buy
#                         print("buy signal and first entry")
                        temp_hold = df_stock['Adj Close'][i+j]
#                         print("Stock:",df_stock['Symbol'][i+j], "price",temp_hold)
                        num+=1
                        long = True
                    # have a short position and close the short position
                    elif short: 
                        total_pnl += temp_hold + df_stock['Adj Close'][i+j]
#                         if symbol=='MRNA':
#                             print("total_pnl", total_pnl,temp_hold/num_s,df_stock['Adj Close'][i+j],num_s)
                        short = False
                        num_s = 0 
                    elif long and not short: 
                        num+=1
                        
                        temp_hold += df_stock['Adj Close'][i+j]
#                         print("long again with price in avg: ",temp_hold//num, 'Times of longs #', num)
                        
                elif df_stock['Sentiment_Vader'][i+j] <= -0.05:
                    # have a long position and close the long position 
                    if long and not short: 
#                         print("sell signal and exit")
                        total_pnl += df_stock['Adj Close'][i+j] -  (temp_hold/num)
                        long = False
                        num = 0 
                    # do not have long or short position
                    elif not long and not short: 
                        temp_hold = - df_stock['Adj Close'][i+j]
                        short = True
                        num_s = 1
    #                 # do not have a long position and already short a stock
                    elif not long and short: 
#                         print("short again")
                        temp_hold = (- df_stock['Adj Close'][i+j]+ temp_hold)/(num_s+1)
                        num_s +=1
#             if symbol=='MRNA':
#                 print('MRNA',df_stock['Sentiment_Vader'][i+j],total_pnl,long,short,temp_hold)
    #             print(j)
            if long and not short and j == n-1: 
                total_pnl += df_stock['Adj Close'][i+j] - (temp_hold/num)
    #             print("Stock:",df_stock['Symbol'][i+j] ,'long: sell on Friday!', 'Entry price', temp_hold,
    #                   'Exit price',df_stock['Adj Close'][i+j], "Total profit:",df_stock['Adj Close'][i+j] - temp_hold  )
            elif short and not long and j == n-1: 
                total_pnl += (df_stock['Adj Close'][i+j] + temp_hold)/(num_s+1)
        stock_names.append(df_stock['Symbol'][i+j])
        len_tweets.append(total_t)
            # long position but no sell signal
    #         print(j)
#         print("long:", long, "short", short )
#         print(pnl)
        pnl += total_pnl
        stock_pnl.append(total_pnl)
        stock = sorted(set(df_stock['Symbol']))
        df_pnl = pd.DataFrame({'Symbol': stock_names, 'PNL_VADER':stock_pnl, 'Len_Ts':len_tweets} )
        
    return pnl, df_pnl

            
            
    

In [309]:
cd ..

/Users/herculesgao/Desktop/Information_Tracer


In [310]:
cd week2

/Users/herculesgao/Desktop/Information_Tracer/week2


In [311]:
ls

SPY12_04_12_08_VADERPNL.csv  SPY500_stock_2023_12_08.csv
SPY500_stock_2023_12_04.csv  SPY500_stock_week1.csv
SPY500_stock_2023_12_05.csv  SPYweek1_VADERPNL.csv
SPY500_stock_2023_12_06.csv  Summary/
SPY500_stock_2023_12_07.csv


In [312]:
import glob
import pandas as pd

glued_data = pd.DataFrame()
n = 0 
for file_name in glob.glob('SPY500_stock_*12*.csv'):
    n +=1
    x = pd.read_csv(file_name, low_memory=False)
    glued_data = pd.concat([glued_data,x],axis=0)
print("number of days", n)

number of days 5


In [313]:
glued_data.sort_values(['Symbol', 'Date'], inplace=True)

In [314]:
glued_data = glued_data.reset_index(drop = True)

In [315]:
dic = {}
for i in glued_data['Symbol']:
    if i not in dic: 
        dic[i] = 1
    else: 
        dic[i] += 1 

In [316]:
[i for i in dic if dic[i]!=n]

[]

In [317]:
if [i for i in dic if dic[i]!=n]:

    glued_data = glued_data[glued_data['Symbol']!=[i for i in dic if dic[i]!=n][0]].reset_index(drop= True)

In [318]:
glued_data['Sentiment_Vader'].isnull().sum()

0

In [319]:
pnl, df_pnl = calculate_total_pnl(glued_data,n)

WM nan
WMB nan


In [320]:
pnl

267.4704502900436

In [300]:
df_pnl.to_csv("SPYweek2_VADERPNL.csv", index=False)

I cannot retrieve the historical implied volatility from SPY. Next week, I am going to add implied vol column every day. 

In [301]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})
table.findAll('tr')[1:][0].findAll('td')[1].text
tickers = []
sectors = []
dc = {}
# print(row.findAll('td')[3].text)
for row in table.findAll('tr')[1:]:
#     if  row.findAll('td')[0].text.replace('\n', '') not in list(df_pnl['Symbol']):
    ticker = row.findAll('td')[0].text
    sector = row.findAll('td')[2].text
    dc[ticker.replace('\n', '').replace('.', '-')] = sector
#     dc[ticker.replace('\n', '')] = company
# start =  datetime.datetime(2023, 12, 1) - timedelta(days = 7)
# end = datetime.datetime(2023, 12, 1)
# data = yf.download(tickers, start=start, end=end)
# df_sector = data.stack().reset_index().rename(index=str, columns={"level_1": "Symbol"}).sort_values(['Symbol','Date'])
# #     df_stock.set_index('Date', inplace=True)
# Sector = [dc[c] for c in dc]
extra_list = []



sectors = []
for i in df_pnl['Symbol']:
    try:
#         print(i,dc[i])
        sectors.append(dc[i])
    except: 
        print("we could not find the sector for the following!",i)
        add_sector = input("Please give a sector for the ticker below!")
        extra_list.append((i,add_sector))
        sectors.append(add_sector)
df_pnl['Sector'] = sectors



we could not find the sector for the following! ALK
Please give a sector for the ticker below!Industrials
we could not find the sector for the following! SEDG
Please give a sector for the ticker below!Information Technology
we could not find the sector for the following! SEE
Please give a sector for the ticker below!Consumer Discretionary


In [302]:
df_pnl['Sector'] = sectors

In [303]:
# [(c, dc[c]) for c in dc]

In [304]:
extra_list

[('ALK', 'Industrials'),
 ('SEDG', 'Information Technology'),
 ('SEE', 'Consumer Discretionary')]

In [305]:
sectors = []
for i in glued_data['Symbol']:
#     print(i,dc[i])
    try: 
        sectors.append(dc[i])
    except: 
        for j in range(len(extra_list)):
            print(extra_list[j])
            if extra_list[j][0] == i: 
#                 print(extra_list[j])

                sectors.append(extra_list[j][1])
                print("found it!")
#                 break

#             print("we could not find the sector for the following!",i)
#             add_sector = input("Please give a sector for the ticker below!")
#             sectors.append(add_sector)
    

('ALK', 'Industrials')
found it!
('SEDG', 'Information Technology')
('SEE', 'Consumer Discretionary')
('ALK', 'Industrials')
found it!
('SEDG', 'Information Technology')
('SEE', 'Consumer Discretionary')
('ALK', 'Industrials')
found it!
('SEDG', 'Information Technology')
('SEE', 'Consumer Discretionary')
('ALK', 'Industrials')
found it!
('SEDG', 'Information Technology')
('SEE', 'Consumer Discretionary')
('ALK', 'Industrials')
found it!
('SEDG', 'Information Technology')
('SEE', 'Consumer Discretionary')
('ALK', 'Industrials')
('SEDG', 'Information Technology')
found it!
('SEE', 'Consumer Discretionary')
('ALK', 'Industrials')
('SEDG', 'Information Technology')
found it!
('SEE', 'Consumer Discretionary')
('ALK', 'Industrials')
('SEDG', 'Information Technology')
found it!
('SEE', 'Consumer Discretionary')
('ALK', 'Industrials')
('SEDG', 'Information Technology')
found it!
('SEE', 'Consumer Discretionary')
('ALK', 'Industrials')
('SEDG', 'Information Technology')
found it!
('SEE', 'Consu

In [306]:
glued_data['Sector'] = sectors

In [307]:
df_pnl.to_csv("SPYweek1_VADERPNL.csv", index=False)
glued_data.to_csv('SPY500_stock_week1.csv', index=False)  